
## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


### К**раткое описание**

- Целью проекта является найти причины, почему не возвращают кредиты в срок. На вход в проект получены данные от банка, которые нужно очистить, обогатить, и предоставить в удобно воспринимаемом виде.
- Не обудем забывать о технической строне, обратим внимание на типы данных. Нужно чтобы наши рассчеты не забирали много вычислительных ресурсов.

На вход получены следующие данные:

Описание данных
-	children — количество детей в семье
-	days_employed — общий трудовой стаж в днях
-	dob_years — возраст клиента в годах
-	education — уровень образования клиента
-	education_id — идентификатор уровня образования
-	family_status — семейное положение
-	family_status_id — идентификатор семейного положения
-	gender — пол клиента
-	income_type — тип занятости
-	debt — имел ли задолженность по возврату кредитов
-	total_income — ежемесячный доход
-	purpose — цель получения кредита

Получить корректный результат после анализа данных можно только на даннах без выбросов, пропущенных значений, имеющих соответствующую размерность. Поэтому мы сначала оценим нашу выборку, посмотрим пропуски, проверим значения на адекватность в каждом столбце. Далее применим методы работы с текстом и выделим категории из текста. Ну и в конце посмотрим как люди вовращают кредиты.

### Шаг 1. Откроем файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
data_set = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')
data_set.info()
display(data_set.tail(15))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21510,2,NaN,28,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,приобретение автомобиля
21511,0,-612.569129,29,высшее,0,гражданский брак,1,F,сотрудник,1,140068.472941,покупка жилья для сдачи
21512,0,-165.377752,26,высшее,0,Не женат / не замужем,4,M,компаньон,0,147301.457769,получение дополнительного образования
21513,0,-1166.216789,35,среднее,1,женат / замужем,0,F,сотрудник,0,250986.142309,покупка жилья
21514,0,-280.469996,27,неоконченное высшее,2,Не женат / не замужем,4,M,компаньон,0,355988.407188,строительство недвижимости
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости


### Вывод

Для анализа надежности заемщиков представлен датасет со статистикой платежеспособности клиентов размерностью 12 столбцов и 21,5 тысяч строк. 

Рассмотрим, какие данные содержит наш *data_set*:
*	education — уровень образования клиента
*	education_id — идентификатор уровня образования
*	family_status — семейное положение
*	family_status_id — идентификатор семейного положения
*	gender — пол клиента
*	income_type — тип занятости
*	debt — имел ли задолженность по возврату кредитов
*	total_income — ежемесячный доход
*	purpose — цель получения кредита
 

Количество значений в столбцах различается. Это говорит о том, что в данных есть пропущенные значения. В столбцах **days_employed** и **total_income** меньше значений, чем в других столбцах, и оно одинаково. В дальнейшем обратим на это внимание.

Наблюдаются несколько типов столбцов *object*, которые потенциально могут быть предметом категоризации заемщиков для оценки их кредитоспрособности.

Попутно попытаемся прикинуть потенциальные количественные метрики, которые можно получить из столбцов с численным значением.

### Шаг 2. Предобработка данных

### Обработка пропусков

В столбцах **days_employed** и **total_income** обнаружены пропущенные значения.

Перед нами встает выбор - заполнить пропущенные значения на основе имеющихся данных, удалить все строки с пропущенными значениями. Так же возможна комбинация этих методов.

Можно попробовать предположить причину, почему эти данные были пропущены:
* Это необязательное поле, оператор пропускал его заполнение
* Человек нехотел указывать информацию по каким то причинам 
* Потеря данных по технической причине (выгрузка БД, некорректная работа приложения, через которое идет ввод данных)
* Различные источники данных в котором небыло данного поля.

Первые две гипотизы не подтверждаются, т.к. количество пропущенных значений было бы различным. А вот вторые 2 вполне реальны.

Исходя из предметной области  столбцы **days_employed** и **total_income** являются значимыми при оценке заемщика и мы не можем игнорировать отсутствие значений в них.

Оценим объем пропусков относительно всего датасэта





In [2]:
#посмотрим на пропуски в датасэте 
print(data_set.isna().sum()) 

#Оценим объем пропусков относительно всего датасэта
data_set_size = data_set.shape[0]
missing_value_score =data_set.isna().sum()[1]
data_set_gap_ratio = missing_value_score / data_set_size
print("Количество данных с пропусками составляет: {:.1%}".format(data_set_gap_ratio))

    

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64
Количество данных с пропусками составляет: 10.1%


10% - не малый объем. Решение о заполнении прмем позже. Сейчас переменуем столбцы для более удобной работы.


In [3]:
#исправим опечатки в названни столбцов **dob_years** на **years** и  **debt** на **debit** , days_employed на years_employed
data_set.set_axis(['children', 'years_employed', 'years', 'education', 'education_id','family_status', 'family_status_id', 'gender', 'income_type', 'debit','monthly_income', 'purpose'],axis = 'columns',inplace = True)
data_set.columns

Index(['children', 'years_employed', 'years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debit',
       'monthly_income', 'purpose'],
      dtype='object')

In [4]:
#Заметим аномальные значения в столбце 'years_employed'
#удалим выборосы по количеству отработанных лет. Предварительно проанализировав, какие данные мы удаляем
# Будем руководствоваться тем, что человек начинает работать с 14 лen
data_set_employ_emission = data_set[data_set['years_employed'] > (data_set['years']-14)]
# посмотрим медианные значения в разных срезах, увидим что почти все люди в выборке - Пенисонеры. У которых может потенциально упасть доход.
def display_category(axis):
    display(data_set_employ_emission.groupby(axis).median())
    df = data_set_employ_emission[axis].value_counts().to_frame()
    df['percent'] = ((data_set_employ_emission[axis].value_counts())*100/data_set_employ_emission.shape[0]).round(1) 
    display(df)
    #display(type(data_set['children'].value_counts()))
columnes = ['children','education','family_status','education','gender','income_type','debit','education',]
for column in columnes:
    display_category(column)
#Удалим часть датасэта с нерелевантными данными и пенсионерами
data_set = data_set.drop(data_set[data_set['years_employed'] > (data_set['years']-14)].index)  

#не будем тратить ресурсы, удалим ненужый df
data_set_employ_emission = None 

data_set.shape

,years_employed,years,education_id,family_status_id,debit,monthly_income
children,,,,,,
-1,350340.760224,54.0,1.0,2.0,0.0,116521.045858
0,365731.749999,60.0,1.0,0.0,0.0,117445.994684
1,359159.819525,58.0,1.0,0.0,0.0,124431.078983
2,371335.674045,52.0,1.0,0.0,0.0,143707.112331
3,348469.112681,42.5,1.0,0.0,0.0,118458.929931
4,399406.269136,46.0,1.0,0.0,0.0,153263.850194
20,360485.069162,61.0,1.0,0.0,0.0,138579.082863


,children,percent
0,3154,91.6
1,253,7.3
2,17,0.5
-1,7,0.2
20,7,0.2
3,6,0.2
4,1,0.0


,children,years_employed,years,education_id,family_status_id,debit,monthly_income
education,,,,,,,
ВЫСШЕЕ,0.0,367532.282115,60.0,0.0,0.0,0.0,165563.164062
Высшее,0.0,371742.501876,56.0,0.0,1.0,0.0,156055.460854
НАЧАЛЬНОЕ,0.0,360264.985350,61.0,3.0,0.0,0.0,90479.336389
НЕОКОНЧЕННОЕ ВЫСШЕЕ,0.0,358732.475441,64.0,2.0,0.0,0.0,412386.347106
Начальное,0.0,364632.447491,59.5,3.0,2.0,0.0,98853.333449
Неоконченное высшее,0.0,396568.887159,60.0,2.0,1.0,0.0,83625.583859
СРЕДНЕЕ,0.0,363541.465259,60.0,1.0,0.0,0.0,113656.027279
Среднее,0.0,363995.316389,60.0,1.0,0.0,0.0,128155.229215
Ученая степень,0.0,337584.815560,69.0,4.0,0.0,0.0,98752.495442


,education,percent
среднее,2511,72.9
высшее,471,13.7
Среднее,148,4.3
СРЕДНЕЕ,135,3.9
начальное,82,2.4
неоконченное высшее,31,0.9
ВЫСШЕЕ,29,0.8
Высшее,21,0.6
НАЧАЛЬНОЕ,7,0.2
Начальное,4,0.1


,children,years_employed,years,education_id,family_status_id,debit,monthly_income
family_status,,,,,,,
Не женат / не замужем,0,367053.055831,60,1,4,0,119038.218597
в разводе,0,366355.800736,60,1,3,0,120388.967772
вдовец / вдова,0,364839.427296,61,1,2,0,117382.901418
гражданский брак,0,366791.950708,59,1,1,0,117711.596792
женат / замужем,0,364225.371849,59,1,0,0,119218.444568


...

,children,years_employed,years,education_id,family_status_id,monthly_income
debit,,,,,,
0,0,365286.622650,60,1,0,117894.955418
1,0,363068.536445,59,1,0,126398.173819


,debit,percent
0,3263,94.7
1,182,5.3


,children,years_employed,years,education_id,family_status_id,debit,monthly_income
education,,,,,,,
ВЫСШЕЕ,0.0,367532.282115,60.0,0.0,0.0,0.0,165563.164062
Высшее,0.0,371742.501876,56.0,0.0,1.0,0.0,156055.460854
НАЧАЛЬНОЕ,0.0,360264.985350,61.0,3.0,0.0,0.0,90479.336389
НЕОКОНЧЕННОЕ ВЫСШЕЕ,0.0,358732.475441,64.0,2.0,0.0,0.0,412386.347106
Начальное,0.0,364632.447491,59.5,3.0,2.0,0.0,98853.333449
Неоконченное высшее,0.0,396568.887159,60.0,2.0,1.0,0.0,83625.583859
СРЕДНЕЕ,0.0,363541.465259,60.0,1.0,0.0,0.0,113656.027279
Среднее,0.0,363995.316389,60.0,1.0,0.0,0.0,128155.229215
Ученая степень,0.0,337584.815560,69.0,4.0,0.0,0.0,98752.495442


,education,percent
среднее,2511,72.9
высшее,471,13.7
Среднее,148,4.3
СРЕДНЕЕ,135,3.9
начальное,82,2.4
неоконченное высшее,31,0.9
ВЫСШЕЕ,29,0.8
Высшее,21,0.6
НАЧАЛЬНОЕ,7,0.2
Начальное,4,0.1


(18080, 12)

In [5]:
#число дней работы не может быть отридцательным, возьмем их по модулю
data_set['years_employed'] = data_set['years_employed'].abs()
# поменяем размерность в трудовом стаже, приведем его к годам
data_set['years_employed'] = (data_set['years_employed']/365).round(decimals =1)
#переменуем столбец 



Мы поработали со столюцом **days_employed** 
- взяли взяли значению по модулю, т.к. была ошибка ввода или выгрузки данных
- убрали ошибочную размерность в часах
- заменили дани **days_employed**  на годы на **years_employed** 


In [6]:

# оценим еще раз таблицу после приведения размерности еще раз,
# заметим в поле children значения -1 и 20 ---- будем разбираться почему так ----
print(data_set['children'].unique())
print(data_set['children'].value_counts())

#будем считать что -1 это ошибка при выгрузке БД. Возьмем значение столб
data_set['children'] = data_set['children'].abs()

print(data_set['children'].value_counts())
#предположим, что значение 20 - было ошибкой при вводе/ выгрузке данных и заменим его на 2
#данное решение не однозначно, т.к. мы не знаем причину, почему получилось данное значение
data_set [data_set['children'] == 20] = 2



[ 1  0  3  2 -1  4 20  5]
 0     10995
 1      4565
 2      2038
 3       324
 20       69
-1        40
 4        40
 5         9
Name: children, dtype: int64
0     10995
1      4605
2      2038
3       324
20       69
4        40
5         9
Name: children, dtype: int64


Поработаем со столбцом **gender**

In [7]:
#проверим на адекватность значения данного столбца
print(data_set['gender'].value_counts())
#убедимся что все строки со значением 2 - это мусор
# одна строка со значением "XNA" не сделает погоды, поэтму удалим и ее
display(data_set[data_set['gender'] == 2].head(5))
data_set = data_set[(data_set['gender'] == 'M')|(data_set['gender'] == 'F')]
#data_set = data_set[data_set.origin.isin(["M", "M"])]
display(data_set['gender'].value_counts())

F      11388
M       6622
2         69
XNA        1
Name: gender, dtype: int64


,children,years_employed,years,education,education_id,family_status,family_status_id,gender,income_type,debit,monthly_income,purpose
606,2,2.0,2,2,2,2,2,2,2,2,2.0,2
720,2,2.0,2,2,2,2,2,2,2,2,2.0,2
1074,2,2.0,2,2,2,2,2,2,2,2,2.0,2
2510,2,2.0,2,2,2,2,2,2,2,2,2.0,2
2941,2,2.0,2,2,2,2,2,2,2,2,2.0,2


F    11388
M     6622
Name: gender, dtype: int64

In [8]:
# Изменим размерность, для наглядность доход будем отображать в тыс. руб
data_set['monthly_income'] = (data_set['monthly_income']/1000).round(decimals =1)
#display(data_set.tail(50))

Cтолбцы  **days_employed** и **total_income** являются значимыми и мы не можем их заполнить их релевантными данными. На самом деле, я бы удалил значения с данными пропусками, но задание требует заполнить.
Примем решение об заполнении пропущенных данных медианным по столбу **income_type*
 

In [9]:
#заполним строки пропущенными значениями медианой по группам из каждого типа доходов
#median_dict = {} # слварь для просмотра "тип дохода":"медиана"
display(data_set.isnull().sum())
#заметим, что у пенсионеров остались записи где отстуствует доход и стаж. Примем решение об удалении всех пенсионеров
display(data_set[data_set['income_type'] == 'пенсионер'])
data_set = data_set.drop(data_set[data_set['income_type'] == 'пенсионер'].index)  

#функция рассчета медианы дохода
def count_median_income(_income_type):
    return (data_set[data_set['income_type'] == _income_type]['monthly_income']).median().round(1)
#функция рассчета медианы времени работы
def count_median_employed(_income_type):
    return (data_set[data_set['income_type'] == _income_type]['years_employed']).median().round(1)
     
#берем каждую группу с типом дохода и присваеваем ей медианное значение в столбцах  'days_employed' и    'total_income'
print(data_set['income_type'].unique())
for i in data_set['income_type'].unique():
   # median_dict[i] = count_median_income(i)
    data_set.loc[(data_set['income_type'] == i)&(data_set['monthly_income'].isna()), 'years_employed'] = count_median_employed(i)
    data_set.loc[(data_set['income_type'] == i)&(data_set['monthly_income'].isna()), 'monthly_income'] = count_median_income(i)
#print(median_dict)   

#убедимся что пропущенные значения отсутствуют
#display(data_set.head(30))

display(data_set.isnull().sum())

children               0
years_employed      2165
years                  0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debit                  0
monthly_income      2165
purpose                0
dtype: int64

,children,years_employed,years,education,education_id,family_status,family_status_id,gender,income_type,debit,monthly_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
145,0,NaN,62,среднее,1,женат / замужем,0,M,пенсионер,0,NaN,строительство недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
21311,0,NaN,49,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для сдачи
21321,0,NaN,56,Среднее,1,женат / замужем,0,F,пенсионер,0,NaN,операции с недвижимостью
21414,0,NaN,65,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,покупка своего жилья
21415,0,NaN,54,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,операции с жильем


['сотрудник' 'компаньон' 'госслужащий' 'предприниматель' 'студент'
 'в декрете']


children            0
years_employed      0
years               0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debit               0
monthly_income      0
purpose             0
dtype: int64

Рассмотрим поля с категориальными значениями

In [10]:
# в 'education' есть проблемы. В 'education_id' 4 категории,
#а в 'education' их больше. Скорее всего понадобится стемминг
print(data_set['education'].unique())
print(data_set['education_id'].unique())

# В столбце с типом заемщика данных проблем нет
print(data_set['family_status'].value_counts())
print(data_set['family_status_id'].value_counts())


# В столбце с типом заемщика данных  нет проблем 
print(data_set['income_type'].unique())

# В целях кредита так же стоит проблема уникальности категоризированных 
#данных которая может потенциально решиться Леммингом
print(data_set['purpose'].unique())

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'УЧЕНАЯ СТЕПЕНЬ' 'ученая степень']
[0 1 2 3 4]
женат / замужем          10247
гражданский брак          3505
Не женат / не замужем     2452
в разводе                  974
вдовец / вдова             421
Name: family_status, dtype: int64
0    10247
1     3505
4     2452
3      974
2      421
Name: family_status_id, dtype: int64
['сотрудник' 'компаньон' 'госслужащий' 'предприниматель' 'студент'
 'в декрете']
['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'операции с жильем' 'образование' 'на проведение свадьбы'
 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку своего автомобиля'
 'на покупку подержанного автомобиля' 'сыграть свадьбу' 'жилье'


### Вывод
Проделана работа по предобработке данных, рассмотрен и оценен каждый столбец, и были выполнены следующие действия:
- заменены значения у строк с пропущенными значениями в столбцах  **days_employed** и **total_income**. Разделили таблицу на группы по типу доходов и заполнили медианным значением в каждой группе.
- удалены строки в столбце **gender** c с нерелевантными значениями **2 и "XNA"**. Восстановление занчений не представляется возможным.
- изменены размерности в столбцах **days_employed** и **total_income**. Теперь трудовой стаж отображается в годах а зарплата в тысячах рублей соответственно.
- в стобцах **days_employed** и **children** взят модуль от значений, т.к. данные были выгружены не корректно. В столбце **children** проведена замена значения '20' на '2', т.к. в этих строках оператор случайно добавлял при вводе или в тот момент некорректно работала форма в ПО, где работал оператор.
- обнаружили необходимость применения стемминга и Лемматизации в столбцах **'education'** и **'purpose'**
- исправлена опечатка в названии столбца **dob_years** на **years** и столбца **debt** на **debit** .
- сбивает с мысли название **total_income**, хотя по факту это ежемесячный доход **monthly_income**
- удаилили 3800 строк с нерелевантными данными в столбце **'years_employed'**, в этих данных в основном Пенисонеры, данные по которым не возможно анализировать в силу некорректно заполненного трудового стажа и показателей дохода.



### Замена типа данных

Произведем замену типов данных в соответствии с таблицей представленной ниже

In [11]:
#Проверим еще раз содержание столбцов на возможность приведения типов
#посмотрим по каждому столбцу уникальные значения 
#заодно всмотримся в цифры, возможно придут какие-то мысли
for column in data_set.columns:
    print(data_set[column].value_counts().sort_values())

5        9
4       40
3      324
2     2035
1     4581
0    10610
Name: children, dtype: int64
39.0       1
36.3       1
50.4       1
39.5       1
35.6       1
        ... 
1.2      230
0.7      231
0.5      255
0.6      295
4.3     1758
Name: years_employed, Length: 392, dtype: int64
75      1
73      1
74      2
72      4
71      5
69      8
70      9
68     12
19     14
67     20
66     26
65     41
20     51
64     57
63     59
0      80
62     89
60    107
21    110
61    113
59    167
22    182
57    210
58    221
23    253
24    261
55    266
56    274
54    315
53    338
25    356
51    364
52    377
26    405
50    445
46    458
47    467
49    474
45    480
27    488
43    502
28    502
32    505
48    514
37    528
44    535
30    537
29    543
36    548
31    556
39    567
33    577
42    581
38    588
40    597
34    597
41    598
35    614
Name: years, dtype: int64
УЧЕНАЯ СТЕПЕНЬ             1
ученая степень             3
Начальное                 10
НАЧАЛЬНОЕ            

In [12]:
#осуществим преобразование тпиов данных. 
#После преобразования наши данные будут обрабатываться быстрее 
#и к ним можно будет применить корректные методы, соответствующие их типам
try:
    data_set['children'] = data_set['children'].astype('uint8')
    data_set['years_employed'] = (data_set['years_employed'].astype('float16'))
    data_set['years'] = data_set['years'].astype('uint16')
    data_set['education'] = data_set['education'].astype('category')
    data_set['education_id'] = data_set['education_id'].astype('uint16')
    data_set['family_status'] = data_set['family_status'].astype('category')
    data_set['family_status_id'] = data_set['family_status_id'].astype('uint16')
    data_set['gender'] = data_set['gender'].astype('category')
    data_set['income_type'] = data_set['income_type'].astype('category')
    data_set['debit'] = data_set['debit'].astype('bool')
    data_set['monthly_income'] = data_set['monthly_income'].astype('float16')                                                               
    data_set['purpose'] = data_set['purpose'].astype('category')
except:
    print('Ошибка! Не удалось осуществить преобразование типов. Проверьте соответствие существующих данных и их типов')
data_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17599 entries, 0 to 21524
Data columns (total 12 columns):
children            17599 non-null uint8
years_employed      17599 non-null float16
years               17599 non-null uint16
education           17599 non-null category
education_id        17599 non-null uint16
family_status       17599 non-null category
family_status_id    17599 non-null uint16
gender              17599 non-null category
income_type         17599 non-null category
debit               17599 non-null bool
monthly_income      17599 non-null float16
purpose             17599 non-null category
dtypes: bool(1), category(5), float16(2), uint16(3), uint8(1)
memory usage: 432.4 KB


In [13]:
# Приведем значения столбца 'education' к нижнему регистру. Это позволит нам однозначно ассоциировать данные с определенной категорией
data_set['education'] = data_set['education'].str.lower()
data_set['education'] = data_set['education'].astype('category')
#проверим что показания 'education_id' и 'education' соответствуют друг другу
print(data_set['education'].value_counts())
print(data_set['education_id'].value_counts())

среднее                12044
высшее                  4668
неоконченное высшее      700
начальное                183
ученая степень             4
Name: education, dtype: int64
1    12044
0     4668
2      700
3      183
4        4
Name: education_id, dtype: int64


In [14]:
#посмотрим визуально на данные, поймем что мы ничто не потеряли после преобразования типов
data_set['years_employed'] = data_set['years_employed'].round(decimals=1)
display(data_set.head(10))


,children,years_employed,years,education,education_id,family_status,family_status_id,gender,income_type,debit,monthly_income,purpose
0,1,23.093750,42,высшее,0,женат / замужем,0,F,сотрудник,False,253.875,покупка жилья
1,1,11.000000,36,среднее,1,женат / замужем,0,F,сотрудник,False,112.125,приобретение автомобиля
2,0,15.398438,33,среднее,1,женат / замужем,0,M,сотрудник,False,145.875,покупка жилья
3,3,11.296875,32,среднее,1,женат / замужем,0,M,сотрудник,False,267.500,дополнительное образование
5,0,2.500000,27,высшее,0,гражданский брак,1,M,компаньон,False,255.750,покупка жилья
6,0,7.898438,43,высшее,0,женат / замужем,0,F,компаньон,False,240.500,операции с жильем
7,0,0.399902,50,среднее,1,женат / замужем,0,M,сотрудник,False,135.750,образование
8,2,19.000000,35,высшее,0,гражданский брак,1,F,сотрудник,False,95.875,на проведение свадьбы
9,0,6.000000,41,среднее,1,женат / замужем,0,M,сотрудник,False,144.375,покупка жилья для семьи
10,2,11.398438,36,высшее,0,женат / замужем,0,M,компаньон,False,113.875,покупка недвижимости


### Вывод

Вполне возможно что со временем на вход нашему пайплайну будут подаваться новые данные, поэтому мы уменьшили диапазон значений числовых переменных в тех столбцах где это возможно. Это существенно прибавит производительности при дальнейших рассчетах.

Если взять примитивный benchmark df.info() то в нашем случае мы получили 432 Kb против 1,7 Мb. Т.е. на данном объеме выборки время вычислений уменьшилось в 4 раза. 

Тип **object** был поменян на тип **category** только в одном столбце, в остальных двух поменяем тип после предобработки категорий и объединения одинаковых по смыслу категорий, но написанных с различными вариациями.


### Обработка дубликатов

In [15]:
#оценим количество дубликатов в нашей выборке
display(data_set[data_set.duplicated()])
data_set = data_set.drop_duplicates().reset_index(drop = True)

#проверим как отработал наш алгоритм
display(data_set[data_set.duplicated()])

,children,years_employed,years,education,education_id,family_status,family_status_id,gender,income_type,debit,monthly_income,purpose
2849,0,4.300781,41,среднее,1,женат / замужем,0,F,сотрудник,False,142.625,покупка жилья для семьи
4182,1,4.300781,34,высшее,0,гражданский брак,1,F,сотрудник,False,142.625,свадьба
6312,0,4.300781,30,среднее,1,женат / замужем,0,M,сотрудник,False,142.625,строительство жилой недвижимости
9238,2,4.300781,34,среднее,1,женат / замужем,0,F,сотрудник,False,142.625,покупка жилья для сдачи
10697,0,4.300781,40,среднее,1,гражданский брак,1,F,компаньон,False,172.375,сыграть свадьбу
11791,0,4.300781,47,среднее,1,женат / замужем,0,F,сотрудник,False,142.625,строительство недвижимости
12373,0,4.300781,58,среднее,1,женат / замужем,0,M,сотрудник,False,142.625,покупка недвижимости
12375,1,4.300781,37,среднее,1,женат / замужем,0,F,сотрудник,False,142.625,покупка недвижимости
13025,1,4.300781,44,среднее,1,женат / замужем,0,F,сотрудник,False,142.625,сделка с подержанным автомобилем
13773,0,4.300781,35,среднее,1,гражданский брак,1,F,сотрудник,False,142.625,сыграть свадьбу


,children,years_employed,years,education,education_id,family_status,family_status_id,gender,income_type,debit,monthly_income,purpose


### Вывод


В нашей выборке был найдено 72 дубликата , что говорит о том что в части ввода данных и работе ПО присутствовали систематические сбои по задвоению записей. Но скорее всего это могло произойти из-за заполнения пропусков.

### Лемматизация

Для задачи категоризации данных используем лемматизацию. Необходимо выделить ключевые слова(леммы) из столбца **purpose** чтобы в конечном итоге однозначно категоризировать клиентов по данному признаку

In [16]:
## не получилось сделать по аналогичной причине, не могу корректно записать в Series
# импортируем библиотеку pymystem3 :
from pymystem3 import Mystem
m = Mystem()
#осуществим лемматизацию, убрав склнения, падежи для дальнейшей категоризации значений столбца
def lemma1(string):
    lemmas =' '.join(m.lemmatize(string))
    return lemmas 
data_set['lemmas'] = data_set['purpose'].apply(lemma1)  


purpose_list = ['жилье','автомобиль','образование','свадьба','недвижимость']
def _value(string1):
    string1 = string1.split(' ')
    for i in string1:
        if i in purpose_list:
            return i
data_set['purpose_category'] = data_set['lemmas'].apply(_value) 
data_set['purpose_category'] = data_set['purpose_category'].astype('category')
data_set.drop(['lemmas'], axis='columns', inplace=True)

#удалим тоже лишние столбцы, которые были нужны на этапе предобработки данных
#сейчас уже в них нету смысла
data_set.drop(['family_status_id'], axis='columns', inplace=True)
data_set.drop(['education_id'], axis='columns', inplace=True)
data_set.drop(['purpose'], axis='columns', inplace=True)

### Вывод

Лингвистический анализ стобца **purpose** посзволил нам создать список ключевых слов и категоризовать данные.
Мы применили лемматизацю, собрали все ключевые слова, выявили уникальные и у нас получился следующий массив значенийй 
['жилье','автомобиль','образование','свадьба','недвижимость']

В конечном итоге в столбце **purpose_category** были получены категории целей кредита, которые мы можем применить для дальнейшего анализа



### Категоризация данных

In [17]:
# оценим столбцы с числовыми и категориальными данными
#выделим числовые данныые, далее классифицируем их, присвоив им категории
data_set.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17555 entries, 0 to 17554
Data columns (total 10 columns):
children            17555 non-null uint8
years_employed      17555 non-null float16
years               17555 non-null uint16
education           17555 non-null category
family_status       17555 non-null category
gender              17555 non-null category
income_type         17555 non-null category
debit               17555 non-null bool
monthly_income      17555 non-null float16
purpose_category    17555 non-null category
dtypes: bool(1), category(5), float16(2), uint16(1), uint8(1)
memory usage: 223.9 KB


Оказалось числовые данные могут быть уже категориальными, напирмер количество детей.

Отбросив идентификаторы, в нашем случе требуют категоризации следующие колонки: **years_employed, years, monthly_income**      

In [18]:
#проведем категоризацию по стобцам: years_employed, years, monthly_income
#Будем разделать на категории не по абсолютным значениям, а по медианным значениям кождого столбца. 
# Для каждого столбца выделим 3 категории с коэфициентами медианных значений 0,6 и 1,25
years_employed_median = data_set['years_employed'].median()
years_category_median = data_set['years'].median()
monthly_income_median = data_set['monthly_income'].median()

# классифицируем наших заемщиков по стажу работы 
def years_employed_category(years_employed):
    if years_employed <= 0.6*years_employed_median:
        return 'Начинающий карьеру'
    if years_employed <= 1.25*years_employed_median:
        return 'Средний опыт'
    return 'Опытный сотрудник' 
data_set['years_employed_category'] = data_set['years_employed'].apply(years_employed_category)

# классифицируем наших заемщиков по возврасту 
def years_category(years_category):
    if years_category <= 0.6*years_category_median:
        return 'Молодые люди'
    if years_category <= 1.25*years_category_median:
        return 'Средний возвраст'
    return 'Пожилой' 
data_set['years_category'] = data_set['years'].apply(years_category)

#классифицируем заемщиков по по уровню дохода
def monthly_income_category(monthly_income):
    if monthly_income <= 0.6*monthly_income_median:
        return 'Низкий доход'
    if monthly_income <= 1.25*monthly_income_median:
        return 'Средний доход'
    return 'Высокий доход' 
data_set['monthly_income_category'] = data_set['monthly_income'].apply(monthly_income_category)

#пререведем сразу тим данных в категориальный
data_set['years_employed_category'] = data_set['years_employed_category'].astype('category')
data_set['years_category'] = data_set['years_category'].astype('category')
data_set['monthly_income_category'] = data_set['monthly_income_category'].astype('category')
display(data_set.head())

,children,years_employed,years,education,family_status,gender,income_type,debit,monthly_income,purpose_category,years_employed_category,years_category,monthly_income_category
0,1,23.093750,42,высшее,женат / замужем,F,сотрудник,False,253.875,жилье,Опытный сотрудник,Средний возвраст,Высокий доход
1,1,11.000000,36,среднее,женат / замужем,F,сотрудник,False,112.125,автомобиль,Опытный сотрудник,Средний возвраст,Средний доход
2,0,15.398438,33,среднее,женат / замужем,M,сотрудник,False,145.875,жилье,Опытный сотрудник,Средний возвраст,Средний доход
3,3,11.296875,32,среднее,женат / замужем,M,сотрудник,False,267.500,образование,Опытный сотрудник,Средний возвраст,Высокий доход
4,0,2.500000,27,высшее,гражданский брак,M,компаньон,False,255.750,жилье,Начинающий карьеру,Средний возвраст,Высокий доход


In [19]:
# Сгруппируем все котегориальные данные, покажемы медианы по группам
# так же оценим количество записей в каждой группе и их доли в общей выборке
def display_category(axis):
    display(data_set.groupby(axis).median())
    df = data_set[axis].value_counts().to_frame()
    df['percent'] = ((data_set[axis].value_counts())*100/data_set.shape[0]).round(1) 
    display(df)
    #display(type(data_set['children'].value_counts()))

#посмотрим группированные данные для каждого категориального столбца, попробуем найти инсайты в данных
columnes = ['children','education','family_status','gender','income_type','debit','purpose_category','years_employed_category', 'years_category','monthly_income_category']
for column in columnes:
    display_category(column)

,years_employed,years,debit,monthly_income
children,,,,
0,4.300781,43.0,False,150.375
1,4.300781,37.0,False,146.750
2,4.300781,35.0,False,142.625
3,4.300781,36.0,False,146.000
4,5.101562,34.5,False,150.375
5,4.300781,36.0,False,168.500


,children,percent
0,10579,60.3
1,4571,26.0
2,2032,11.6
3,324,1.8
4,40,0.2
5,9,0.1


,children,years_employed,years,debit,monthly_income
education,,,,,
высшее,0.0,4.300781,37,False,172.375
начальное,0.0,3.599609,39,False,134.000
неоконченное высшее,0.0,3.500000,31,False,159.500
среднее,0.0,4.300781,41,False,142.625
ученая степень,0.5,10.546875,41,False,157.250


,education,percent
среднее,12008,68.4
высшее,4660,26.5
неоконченное высшее,700,4.0
начальное,183,1.0
ученая степень,4,0.0


,children,years_employed,years,debit,monthly_income
family_status,,,,,
Не женат / не замужем,0,3.900391,33,False,147.750
в разводе,0,4.300781,43,False,150.750
вдовец / вдова,0,5.601562,52,False,142.625
гражданский брак,0,4.300781,39,False,147.000
женат / замужем,0,4.300781,40,False,149.500


...

,years_employed_category,percent
Опытный сотрудник,7010,39.9
Средний опыт,5705,32.5
Начинающий карьеру,4840,27.6


,children,years_employed,years,debit,monthly_income
years_category,,,,,
Молодые люди,0,2.400391,22,False,136.000
Пожилой,0,5.500000,54,False,148.125
Средний возвраст,0,4.300781,36,False,150.250


,years_category,percent
Средний возвраст,12874,73.3
Пожилой,3993,22.7
Молодые люди,688,3.9


,children,years_employed,years,debit,monthly_income
monthly_income_category,,,,,
Высокий доход,0,4.800781,40,False,244.3750
Низкий доход,0,4.300781,39,False,73.6875
Средний доход,0,4.300781,39,False,142.6250


,monthly_income_category,percent
Средний доход,9983,56.9
Высокий доход,5338,30.4
Низкий доход,2234,12.7


In [20]:
# посмотрим средний возвраст тех, кто начинает карьеру и какое у них образование
data_pivot = data_set.copy()
data_pivot = data_pivot.pivot_table(index=['years_employed_category'], columns='education', values='years', aggfunc='median')
display(data_pivot.head(10))

education,высшее,начальное,неоконченное высшее,среднее,ученая степень
years_employed_category,,,,,
Начинающий карьеру,34.0,38.0,28.0,38.0,45.0
Опытный сотрудник,41.0,40.0,36.0,43.0,37.0
Средний опыт,35.0,38.0,31.0,39.0,NaN


### Вывод

После категоризации столбцов с числовыми значениями мы обогатили данные и получили дополнительную возможность группировать данные. Эта информация в дальнейшем нам пригодиться при оценке платежеспособности клиентов.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [21]:
#добавим дополнительный столбец по наличию детей
def children_status_category(children_status):
    if children_status == 0:
        return '0'
    else:
        return '1'
data_set['children_status'] = data_set['children'].apply(children_status_category)

data_pivot = data_set.copy()
pivot = data_set.pivot_table(index='children_status', values='debit', aggfunc=['count', 'sum', 'mean'])
pivot['mean'] = (pivot['mean']*100).round(1)
display(pivot)


,count,sum,mean
,debit,debit,debit
children_status,,,
0,10579,865.0,8.2
1,6976,651.0,9.3


### Вывод

 - Проанализировав соотношение должников к исправно платащим мы почти не заметим разницы. Должников без детей **9.3%**, а должников с детьми **8.2%**. Данную гипотезу будем считать неверной


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [22]:
data_pivot = data_set.copy()
pivot = data_set.pivot_table(index='family_status', values='debit', aggfunc=['count', 'sum', 'mean'])
pivot['mean'] = (pivot['mean']*100).round(1)
display(pivot)

,count,sum,mean
,debit,debit,debit
family_status,,,
Не женат / не замужем,2450,257.0,10.5
в разводе,974,71.0,7.3
вдовец / вдова,421,26.0,6.2
гражданский брак,3490,349.0,10.0
женат / замужем,10220,813.0,8.0


### Вывод

Живущие гражданским браком или имеющие статус "женат / замужем" чаще не возвращают кредиты в срок, таких  **10 и 10.5%** соответственно. Остальные категории не сильно отстают среднее значение по ним составляет около **7%**

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [23]:
data_pivot = data_set.copy()
pivot = data_set.pivot_table(index='monthly_income_category', values='debit', aggfunc=['count', 'sum', 'mean'])
pivot['mean'] = (pivot['mean']*100).round(1)
display(pivot)



,count,sum,mean
,debit,debit,debit
monthly_income_category,,,
Высокий доход,5338,404.0,7.6
Низкий доход,2234,200.0,9.0
Средний доход,9983,912.0,9.1


### Вывод

Как ни странно - уровень дохода не влияет на процент возврата по кредиту. У заемщиков с высоким доходом уровень возврата **7.6%**, у заемщиков со средним и низким доходом около **9%**. Люди с высоким доходом реже задерживают или не возвращают выплаты.

- Как разные цели кредита влияют на его возврат в срок?

In [24]:
data_pivot = data_set.copy()
pivot = data_set.pivot_table(index='purpose_category', values='debit', aggfunc=['count', 'sum', 'mean'])
pivot['mean'] = (pivot['mean']*100).round(1)
display(pivot)

,count,sum,mean
,debit,debit,debit
purpose_category,,,
автомобиль,3497,350.0,10.0
жилье,3674,267.0,7.3
недвижимость,5213,420.0,8.1
образование,3277,321.0,9.8
свадьба,1894,158.0,8.3


### Вывод

В срок чаще не возвращают кредиты, которые были взяты с целью образования или покупки автомобиля. Таких заемщиков около **10%**. В категориях недвижимость, жилье и свадьба количество невозвратов колеблется около **8%**

### Шаг 4. Общий вывод

Проделана работа по предобработке данных, рассмотрен и оценен каждый столбец, и были выполнены следующие действия:
- заменены значения у строк с пропущенными значениями в столбцах  **days_employed** и **total_income**. Разделили таблицу на группы по типу доходов и заполнили медианным значением в каждой группе.
- удалены строки в столбце **gender** c с нерелевантными значениями **2 и "XNA"**. Восстановление занчений не представляется возможным.
- изменены размерности в столбцах **days_employed** и **total_income**. Теперь трудовой стаж отображается в годах а зарплата в тысячасх рублей соответственно.
- в стобцах **days_employed** и **children** взят модуль от значений, т.к. данные были выгружены не корректно. В столбце **children** проведена замена значения '20' на '2', т.к. в этих строках оператор случайно добавлял при вводе или в тот момент некорректно работала форма в ПО, где работал оператор.
- обнаружили необходимость применения стемминга и Лемматизации в столбцах **'education'** и **'purpose'**
- исправлена опечатка в названии столбца **dob_years** на **years** и столбца **debt** на **debit** .
- сбивает с мысли название **total_income**, хотя по факту это ежемесячный доход **monthly_income**
- удаилили 3800 строк с нерелевантными данными в столбце **'years_employed'**, в этих данных в основном Пенисонеры, данные по которым не возможно анализировать в силу некорректно заполненного трудового стажа и показателей дохода.

Вполне возможно что со временем на вход нашему пайплайну будут подаваться новые данные, поэтому мы уменьшили диапазон значений числовых переменных в тех столбцах где это возможно. Это существенно прибавит производительности при дальнейших рассчетах.

Если взять примитивный benchmark df.info() то в нашем случае мы получили 432 Kb против 1,7 Мb. Т.е. на данном объеме выборки время вычислений уменьшилось в 4 раза. 

Тип **object** был поменян на тип **category** только в одном столбце, в остальных двух поменяем тип после предобработки категорий и объединения одинаковых по смыслу категорий, но написанных с различными вариациями.

Лингвистический анализ стобца **purpose** посзволил нам создать список ключевых слов и категоризовать данные.
Мы применили лемматизацю, собрали все ключевые слова, выявили уникальные и у нас получился следующий массив значенийй 
['жилье','автомобиль','образование','свадьба','недвижимость']

В конечном итоге в столбце **purpose_category** были получены категории целей кредита, которые мы можем применить для дальнейшего анализа

После категоризации столбцов с числовыми значениями мы обогатили данные и получили дополнительную возможность группировать данные. 

#### Проанализировав как возвращают кредиты мы пришли к следующему выводу:
 - Проанализировав соотношение должников к исправно платащим мы почти не заметим разницы. Должников без детей **9.33%**, а должников с детьми **8.18%**. Данную гипотезу будем считать неверной
 - Живущие гражданским браком или имеющие статус "женат / замужем" чаще не возвращают кредиты в срок, таких около **10 и 10.5%** соответственно. Остальные категории не сильно отстают среднее значение по ним составляет около **7%**
 - Как ни странно - уровень дохода не влияет на процент возврата по кредиту. У заемщиков с высоким доходом уровень возврата **7.6%**, у заемщиков со средним и низким доходом около **9%**. Люди с высоким доходом реже задерживают или не возвращают выплаты.
 - В срок чаще не возвращают кредиты, которые были взяты с целью образования или покупки автомобиля. Таких заемщиков около **10%**. В категориях недвижимость, жилье и свадьба количество невозвратов колеблется около **8%**



### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.